## You may ignore this. This is just for loading the files online. 

## (But you still have to run this [shift+enter])


In [0]:
!wget -O iris_data.csv https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

## Import important packages.

In [0]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
%matplotlib inline

## Load the data

In [0]:
iris = pd.read_csv('iris_data.csv', header = None)
iris.columns = ['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
iris.dropna(how="all", inplace=True) ## no need to drop but for other dataset, you may want this!

## Take a quick glance at the data

In [0]:
iris.head()

In [0]:
X = iris.iloc[:,0:4].values
y = iris.iloc[:,4].values

## Expolatory Visualization

In [0]:
label_dict = {1: 'Setosa',
              2: 'Versicolor',
              3: 'Virgnica'}

feature_dict = {0: 'sepal length',
                1: 'sepal width',
                2: 'petal length',
                3: 'petal width'}

with plt.style.context('bmh'):
    plt.figure(figsize=(8, 6))
    for cnt in range(4):
        plt.subplot(2, 2, cnt+1)
        for lab in ('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'):
            plt.hist(X[y==lab, cnt],
                     label=lab,
                     bins=10,
                     alpha=0.4)
        plt.xlabel(feature_dict[cnt])
    plt.legend(loc='upper right', fancybox=True, fontsize=8)

    plt.tight_layout()
    plt.show()

## Standardize the data before finding covariance matrix.

In [0]:
from sklearn.preprocessing import StandardScaler
X_std_sc = StandardScaler().fit_transform(X)

Covariance matrix formula:

$$ \Sigma = \frac{1}{n-1} (\mathbf{X} - \bar{X})^T(\mathbf{X} - \bar{X})$$

In [0]:
X_mean = np.mean(X_std_sc, axis=0)
X_cov = (X_std_sc - X_mean).T.dot((X_std_sc - X_mean)) / (X_std_sc.shape[0]-1)
print('Covariance matrix \n%s' %X_cov)

We may use a built-in function from numpy to find covariance matrix, eigenvalues, and eigenvectors.

In [0]:
cov_mat = np.cov(X_std_sc.T)

eig_vals, eig_vecs = np.linalg.eig(cov_mat)

print('Covariance matrix: \n%s' %cov_mat)
print('Eigenvectors: \n%s' %eig_vecs)
print('\nEigenvalues: \n%s' %eig_vals)

As in the slides, eigenvalues here are always greater than 0 because the covariance matrix is positive semi-definite. 

## Selecting principal components


The eigen with the lowest value contains the least information. 

However, how do we know which one we can really drop?

In [0]:
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

eig_pairs.sort(key=lambda x: x[0], reverse=True)

In [0]:
total = sum(eig_vals)
var_exp = [(i / total)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

with plt.style.context('bmh'):
    plt.figure(figsize=(6, 4))

    plt.bar(range(4), var_exp, alpha=0.5, align='center',
            label='explained variance')
    plt.step(range(4), cum_var_exp, where='mid',
             label='cumulative explained variance')
    plt.ylabel('Explained Variance Percentage')
    plt.xlabel('Eigenvectors')
    plt.legend(loc='best')
    plt.tight_layout()

In [0]:
var_exp

## Forming a projection matrix

Let's say we would like to drop 2 dimensions since the last 2 eigenvalues only contains 3.5 and 0.5% of the variance. The input should be a 4-dim vector and the output is a 2-dim vecotr. Hence, the projection matrix is supposed to be 4x2 matrix.

In [0]:
eig_pairs

In [0]:
proj_matrix = np.hstack((eig_pairs[0][1].reshape(4,1),
                      eig_pairs[1][1].reshape(4,1)))

print('Projection Matrix :\n', proj_matrix)

## Projection onto a new space

Now, we want to transform 4-dim vector into 2-dim vector, we can project the input with the matrix W above as follows.


$$ Y = X \times W $$

where Y is a 150x2 matrix (150 rows of records, each with 2 features).

In [0]:
Y = X_std_sc.dot(proj_matrix)

with plt.style.context('bmh'):
    plt.figure(figsize=(8,5))
    for lab, col in zip(('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'),
                        ('blue', 'red', 'green')):
        plt.scatter(Y[y==lab, 0],
                    Y[y==lab, 1],
                    label=lab,
                    c=col,)
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.legend(loc='lower center')
    plt.tight_layout()
    plt.show()

## Shortcut in scikit-learn package

In [0]:
from sklearn.decomposition import PCA as sklearnPCA
pca_model = sklearnPCA(n_components=2)
Y_trans = pca_model.fit_transform(X_std_sc)


In [0]:
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(8, 5))
    for lab, col in zip(('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'),
                        ('blue', 'red', 'green')):
        plt.scatter(Y_trans[y==lab, 0],
                    Y_trans[y==lab, 1],
                    label=lab,
                    c=col,)
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.legend(loc='lower center')
    plt.tight_layout()
    plt.show()

## See the differences?

We did that manually. Notice the second axis, if we multiply -1 to the second component, we get the same thing.

In [0]:
second_proj_matrix = np.hstack((eig_pairs[0][1].reshape(4,1),
                      -eig_pairs[1][1].reshape(4,1)))

print('Projection Matrix :\n', second_proj_matrix)

In [0]:
Y = X_std_sc.dot(second_proj_matrix)

with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(8,5))
    for lab, col in zip(('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'),
                        ('blue', 'red', 'green')):
        plt.scatter(Y[y==lab, 0],
                    Y[y==lab, 1],
                    label=lab,
                    c=col,)
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.legend(loc='lower center')
    plt.tight_layout()
    plt.show()

### Try to do PCA by yourself on "sample_data.csv"